# Equity Examples
(c) Marek Ozana

Example usage of polars-bloomberg in Equity space

In [1]:
# imports
import altair as alt

from polars_bloomberg import BQuery


## S&P Annual Price Returns

In [5]:
# get S&P annual price returns
# todo: not exactly correct, since pct_change should include previous period.
query = """
let(
    #year = year(px_last(dates=range(-100Y, 0D), fill=prev, per=W).date);
    #rets = pct_chg(group(px_last(dates=range(-100Y, 0D), fill=prev, per=W),
                          by=#year));
    #bins = bins(#rets,
                 [-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                 ['< -50', '-50 to -40', '-40 to -30', '-30 to -20',
                  '-20 to -10', '-10 to 0', '0 to 10', '10 to 20',
                  '20 to 30', '30 to 40', '40 to 50', '> 50']);
     )
get(#rets, #bins)
for('SPX Index')
"""
with BQuery() as bq:
    df = bq.bql(query).combine()

# Create Chart
base = alt.Chart(df, title="S&P 500 Annual Price Returns").encode(
    x=alt.X("#bins:O", title="Annual Returns (%)").sort(
        [
            "< -50",
            "-50 to -40",
            "-40 to -30",
            "-30 to -20",
            "-20 to -10",
            "-10 to 0",
            "0 to 10",
            "10 to 20",
            "20 to 30",
            "30 to 40",
            "40 to 50",
            "> 50",
        ]
    ),
    y=alt.Y("count()", title="Count").stack("zero"),
)
bars = base.mark_bar().encode(
    color=alt.Color("#YEAR:O").scale(scheme="blueorange", reverse=True).legend(None)
)
txt = base.mark_text(dy=7).encode(text=alt.Text("#YEAR:O"))

(bars + txt).properties(width=600, height=400)

alt.LayerChart(...)